# Parse raw CUSIP data

In [1]:
import pandas as pd
from tqdm import tqdm

def is_cusip(s):    
    q = s[:11]
    if len(q) < 11:
        return False
    if q[6] != ' ':
        return False
    if q[9] != ' ':
        return False    
    if not ('0' <= q[10] <= '9'):
        return False    
    nn = 0
    ns = 0
    for c in q:
        if '0' <= c <= '9':
            nn += 1
        elif 'A' <= c <= 'Z':
            pass
        elif c == ' ':
            ns += 1
        else:
            return False
    return nn > 2 and ns == 2

is_cusip('G02602 95 3'), is_cusip('STATUS'), is_cusip('4')

(True, False, False)

In [2]:
df = pd.read_csv('cusips_raw.tsv', sep='\t')
df

,path,line_number,line
0,source_data/2004q1.pdf,1,List of Section 13F Securities
1,source_data/2004q1.pdf,2,NaN
2,source_data/2004q1.pdf,3,1st Quarter (cid:139) FY 2004
3,source_data/2004q1.pdf,4,NaN
4,source_data/2004q1.pdf,5,Copyright (c) 2004 American Bankers Associatio...
...,...,...,...
5209440,source_data/2017q4.pdf,93566,PUT
5209441,source_data/2017q4.pdf,93567,NaN
5209442,source_data/2017q4.pdf,93568,"Total Count: 17,770"
5209443,source_data/2017q4.pdf,93569,NaN


In [3]:
useless = {'', '*', 'IVM001', 'ISSUER NAME', 'ISSUER DESCRIPTION', 'STATUS'}
lines = [li.strip() for li in df.dropna().line.to_list()]
lines = [li for li in lines if li not in useless]
lines

['List of Section 13F Securities',
 '1st Quarter (cid:139) FY 2004',
 'Copyright (c) 2004 American Bankers Association. CUSIP Numbers and',
 'descriptions are used with permission by Standard & Poors CUSIP Service',
 'Bureau, a division of The McGraw-Hill Companies, Inc.  All rights reserved.',
 'No redistribution without permission from Standard & Poors CUSIP Service',
 'Bureau.',
 'Standard & Poors CUSIP Service Bureau does not guarantee the accuracy',
 'or completeness of the CUSIP Numbers and standard descriptions included',
 'herein and neither the American Bankers Association nor Standard &',
 "Poor's CUSIP Service Bureau shall be responsible for any errors, omissions",
 'or damages arising out of the use of such information.',
 'U.S. Securities and Exchange Commission',
 'OFFICIAL LIST OF SECTION 13(f) SECURITIES USER INFORMATION SHEET',
 'General',
 'This list of “Section 13(f) securities” as defined by Rule',
 '13f-1(c)  [17 CFR 240.13f-1(c)] is made available to the public',


In [4]:
cusips = []
cusip = []
record = False

def add_cusip_to_list():
    if cusip:
        cusips.append(cusip)
    
for li in tqdm(lines):
    isc = is_cusip(li)
    if isc:
        add_cusip_to_list()
        if len(li) == 11:
            cusip = [li]
        else:
            cusip = [li[:11], li[11:]]
        record = True
    elif record:
        if li == 'Run Date:':
            record = False
        else:
            if li not in set(cusip):
                cusip.append(li)

add_cusip_to_list()
cusips

100%|██████████| 2409393/2409393 [00:04<00:00, 505842.15it/s]


[['B49233 10 7', 'ICOS VISION SYS CORP N V'],
 ['D1497A 10 1', 'CELANESE AG'],
 ['D1668R 12 3', 'DAIMLERCHRYSLER AG'],
 ['D1668R 90 3', 'DAIMLERCHRYSLER AG'],
 ['D1668R 95 3', 'DAIMLERCHRYSLER AG', 'ORD', 'CALL', 'PUT'],
 ['D18190 89 8', 'DEUTSCHE BANK AG NAMEN', 'ORD'],
 ['D18190 90 8', 'DEUTSCHE BANK AG NAMEN'],
 ['D18190 95 8', 'DEUTSCHE BANK AG NAMEN', 'CALL', 'PUT'],
 ['G0070K 10 3', 'ACE LTD', 'ORD'],
 ['G0070K 90 3', 'ACE LTD'],
 ['G0070K 95 3', 'ACE LTD', 'CALL', 'PUT'],
 ['G02602 10 3', 'AMDOCS LTD', 'ORD'],
 ['G02602 90 3', 'AMDOCS LTD'],
 ['G02602 95 3', 'AMDOCS LTD'],
 ['G02995 10 1', 'AMERICAN SAFETY INS HLDGS LT'],
 ['G03910 10 9', 'ANNUITY AND LIFE RE HLDGS'],
 ['G03910 90 9', 'ANNUITY AND LIFE RE HLDGS'],
 ['G03910 95 9', 'ANNUITY AND LIFE RE HLDGS', 'CALL', 'PUT', 'ORD'],
 ['G04074 10 3', 'APEX SILVER MINES LTD', 'ORD'],
 ['G04074 90 3', 'APEX SILVER MINES LTD'],
 ['G04074 95 3', 'APEX SILVER MINES LTD', 'CALL', 'PUT'],
 ['G0450A 10 5', 'ARCH CAP GROUP LTD', 'ORD'],
 [

In [5]:
def parse_list(cusip_list):
    cusip = cusip_list[0][:6] + cusip_list[0][7:9]
    name = ' '.join(cusip_list[1:3])
    name = name.split()
    name = [w for w in name if w not in {'', '*', 'DELETED', 'ADDED'}]
    name = ' '.join(name)
    return cusip, name

parse_list(cusips[-1])

('98986X95', 'ZYNERBA PHARMACEUTICALS INC PUT')

In [6]:
m = [parse_list(li) for li in tqdm(cusips)]
m = pd.DataFrame(m, columns='cusip cusipname'.split())
m.shape

100%|██████████| 889160/889160 [00:01<00:00, 512191.95it/s]


(889160, 2)

In [7]:
m = m[m.cusipname.apply(lambda s: 'DELETED' not in s)]
m.shape

(889160, 2)

In [8]:
m.tail()

,cusip,cusipname
889155,98986T90,ZYNGA INC
889156,98986T95,ZYNGA INC
889157,98986X10,ZYNERBA PHARMACEUTICALS INC CALL
889158,98986X90,ZYNERBA PHARMACEUTICALS INC CALL
889159,98986X95,ZYNERBA PHARMACEUTICALS INC PUT


In [9]:
m = m.groupby('cusip', as_index=False)['cusipname'].apply(lambda li: list(li)[-1])
m.shape

(36460, 2)

In [10]:
m[m.cusip == '00030710']

,cusip,cusipname
0,00030710,AAC HLDGS INC


In [11]:
m.to_csv('cusips.tsv', index=False, sep='\t')